In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# https://www.tensorflow.org/guide/keras/overview

# Build a simple model

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
from tensorflow.keras import layers

model=tf.keras.Sequential()

model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10))

In [ ]:
layers.Dense(64, activation='relu')
layers.Dense(64, activation=tf.nn.relu)

layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l1(0.01))

layers.Dense(64, bias_regularizer=tf.keras.regularizers.l2(0.01))

layers.Dense(64, kernel_initializer='orthogonal')

layers.Dense(64, bias_initializer=tf.keras.initializers.Constant(2.0))

In [ ]:
model=tf.keras.Sequential([layers.Dense(64, activation='relu', input_shape=(32,)),
                          layers.Dense(64, activation='relu'),
                          layers.Dense(10)])

model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
             loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
             metrics=['accuracy'])

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.01), 
              loss='mse',
             metrics=['mae'])

model.compile(optimizer=tf.keras.optimizers.RMSprop(0.01), 
             loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
             metrics=['accuracy'])


In [ ]:
import numpy as np
data=np.random.random((1000, 32))
labels=np.random.random((1000, 10))

model.fit(data, labels, epochs=10, batch_size=32)

In [ ]:
import numpy as np

data=np.random.random((1000, 32))
labels=np.random.random((1000, 10))

val_data=np.random.random((100, 32))
val_labels=np.random.random((100, 10))

model.fit(data, labels, epochs=10, batch_size=32, 
         validation_data=(val_data, val_labels))

In [ ]:
dataset=tf.data.Dataset.from_tensor_slices((data, labels))
dataset=dataset.batch(32)
model.fit(dataset, epochs=10)

In [ ]:
dataset=tf.data.Dataset.from_tensor_slices((data, labels))
dataset=dataset.batch(32)

val_dataset=tf.data.Dataset.from_tensor_slices((val_data, val_labels))
val_dataset=val_dataset.batch(32)

model.fit(dataset, epochs=10, validation_data=val_dataset)

In [ ]:
data=np.random.random((1000, 32))
labels=np.random.random((1000, 10))

model.evaluate(data, labels, batch_size=32)

dataset=tf.data.Dataset.from_tensor_slices((data, labels))
dataset=dataset.batch(32)

model.evaluate(dataset)

In [ ]:
result=model.predict(data, batch_size=32)
print(result.shape, type(result))

# Build complex models

In [ ]:
inputs=tf.keras.Input(shape=(32,))

x=layers.Dense(64, activation='relu')(inputs)
x=layers.Dense(64, activation='relu')(x)
predictions=layers.Dense(10)(x)

In [ ]:
model=tf.keras.Model(inputs=inputs, outputs=predictions)

model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
             loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
             metrics=['accuracy'])

model.fit(data, labels, batch_size=32, epochs=5)

In [ ]:
class MyModel(tf.keras.Model):
    def __init__(self, num_classes=10):
        super(MyModel, self).__init__(name='my_model')
        self.num_classes=num_classes
        
        self.dense_1=layers.Dense(64, activation='relu')
        self.dense_2=layers.Dense(64, activation='relu')
        
        self.prediction=layers.Dense(self.num_classes)
        
    def call(self, inputs):
        x=self.dense_1(inputs)
        x=self.dense_2(x)
        return self.prediction(x)

In [ ]:
model=MyModel(num_classes=10)

model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
             loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
             metrics=['accuracy'])

model.fit(data, labels, batch_size=32, epochs=5)

In [ ]:
class MyLayer(layers.Layer):
    
    def __init__(self, output_dim, **kwargs):
        self.output_dim=output_dim
        super(MyLayer, self).__init__(**kwargs)
        
    def build(self, input_shape):
        self.kernel=self.add_weight(name='kernel',
                                   shape=(input_shape[1], self.output_dim),
                                   initializer='uniform',
                                   trainable=True)
        
    def call(self, inputs):
        return tf.matmul(inputs, self.kernel)
    
    def get_config(self):
        base_config=super(MyLayer, self).get_config()
        base_config['output_dim']=self.output_dim
        
        return base_config
    
    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [ ]:
model=tf.keras.Sequential([MyLayer(10)])

model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
             loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
             metrics=['accuracy'])

model.fit(data, labels, batch_size=32, epochs=5)

In [ ]:
callbacks=[
    tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs')
]

model.fit(data, labels, batch_size=32, epochs=5, callbacks=callbacks, 
         validation_data=(val_data, val_labels))

In [ ]:
model=tf.keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(32,)),
    layers.Dense(10)
])

model.compile(optimizer=tf.keras.optimizers.Adam(0.0001),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy']
             )

In [ ]:
model.save_weights('./weights/my_model')
model.load_weights('./weights/my_model')

In [ ]:
model.save_weights('my_model.h5', save_format='h5')
model.load_weights("my_model.h5")

In [ ]:
json_string=model.to_json()
json_string

In [ ]:
import json
import pprint

pprint.pprint(json.loads(json_string))

In [ ]:
fresh_model=tf.keras.models.model_from_json(json_string)

In [ ]:
yaml_string=model.to_yaml()
print(yaml_string)

In [ ]:
fresh_model=tf.keras.models.model_from_yaml(yaml_string)

In [ ]:
# Create a simple model
model = tf.keras.Sequential([
  layers.Dense(10, activation='relu', input_shape=(32,)),
  layers.Dense(10)
])
model.compile(optimizer='rmsprop',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(data, labels, batch_size=32, epochs=5)

model.save('my_model')
model=tf.keras.models.load_model('my_model')

In [ ]:
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
  model = tf.keras.Sequential()
  model.add(layers.Dense(16, activation='relu', input_shape=(10,)))
  model.add(layers.Dense(1))

  optimizer = tf.keras.optimizers.SGD(0.2)

  model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                optimizer=optimizer)

model.summary()

In [ ]:
x = np.random.random((1024, 10))
y = np.random.randint(2, size=(1024, 1))
x = tf.cast(x, tf.float32)
dataset = tf.data.Dataset.from_tensor_slices((x, y))
dataset = dataset.shuffle(buffer_size=1024).batch(32)

model.fit(dataset, epochs=1)

In [ ]:
help(model.summary())